In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

order_df = pd.read_excel("/content/drive/MyDrive/codera/Company X - Order Report.xlsx")
pincode_df = pd.read_excel("/content/drive/MyDrive/codera/Company X - Pincode Zones.xlsx")
invoice_df = pd.read_excel("/content/drive/MyDrive/codera/Courier Company - Invoice.xlsx")
sku_df = pd.read_excel("/content/drive/MyDrive/codera/Company X - SKU Master.xlsx")
rate_df = pd.read_excel("/content/drive/MyDrive/codera/Courier Company - Rates.xlsx")

#Calculate order weight from SKU master
order_df = order_df.merge(sku_df, on='SKU', how='left')
order_df['Order Weight (KG)'] = order_df['Order Qty'] * order_df['Weight (g)'] / 1000
order_wt = order_df.groupby('ExternOrderNo')['Order Weight (KG)'].sum().reset_index()

#Get zone between warehouse and customer pincodes
order_zone = order_wt.merge(pincode_df, left_on='ExternOrderNo', right_on='Customer Pincode', how='left')

In [3]:
print(invoice_df.columns)

Index(['AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode',
       'Customer Pincode', 'Zone', 'Type of Shipment', 'Billing Amount (Rs.)'],
      dtype='object')


In [4]:
result_df = invoice_df.merge(order_zone, left_index=True, right_index=True, how='left')

In [ ]:
print(result_df.columns)

Index(['AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode_x',
       'Customer Pincode_x', 'Zone_x', 'Type of Shipment',
       'Billing Amount (Rs.)', 'ExternOrderNo', 'Order Weight (KG)',
       'Warehouse Pincode_y', 'Customer Pincode_y', 'Zone_y'],
      dtype='object')


In [ ]:
invoice_df['Order_ID'] = invoice_df['Order ID']
order_zone['Order_ID'] = order_zone['ExternOrderNo']

result_df = invoice_df.merge(order_zone, on='Order_ID', how='left')

In [ ]:
#Join Order data with Invoice details
invoice_df['Order_ID'] = invoice_df['Order ID']
order_zone['Order_ID'] = order_zone['ExternOrderNo']

result_df = invoice_df.merge(order_zone, on='Order_ID', how='left')

In [ ]:
result_df

,AWB Code,Order ID,Charged Weight,Warehouse Pincode_x,Customer Pincode_x,Zone_x,Type of Shipment,Billing Amount (Rs.),Order_ID,ExternOrderNo,Order Weight (KG),Warehouse Pincode_y,Customer Pincode_y,Zone_y
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,2001806232,2001806232,95.944,NaN,NaN,NaN
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,2001806273,2001806273,47.972,NaN,NaN,NaN
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,2001806408,2001806408,143.916,NaN,NaN,NaN
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,2001806458,2001806458,23.986,NaN,NaN,NaN
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,2001807012,2001807012,11.993,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,2001812941,2001812941,35.979,NaN,NaN,NaN
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,2001809383,2001809383,35.979,NaN,NaN,NaN
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,2001820978,2001820978,23.986,NaN,NaN,NaN
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,2001811475,2001811475,47.972,NaN,NaN,NaN


In [ ]:
order_zone = order_wt.merge(pincode_df, left_on='ExternOrderNo', right_on='Customer Pincode', how='left')

In [ ]:
print(order_wt.columns)
print(pincode_df.columns)


Index(['ExternOrderNo', 'Order Weight (KG)'], dtype='object')
Index(['Warehouse Pincode', 'Customer Pincode', 'Zone'], dtype='object')


In [ ]:
# Rename columns in order_zone DataFrame to avoid suffixes
order_zone.rename(columns={'Zone': 'Zone_order'}, inplace=True)

# Join Order data with Invoice details
result_df = invoice_df.merge(order_zone, on='Order ID', how='left')

print(result_df.columns)


Index(['AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode_x',
       'Customer Pincode_x', 'Zone', 'Type of Shipment',
       'Billing Amount (Rs.)', 'Order_ID', 'Order Weight (KG)',
       'Warehouse Pincode_y', 'Customer Pincode_y', 'Zone_order'],
      dtype='object')


In [ ]:
# Calculate expected shipment weight slab and charges for Company X
def get_weight_slab(row):
    if row['Zone_order'] == 'a':
        return round(row['Order Weight (KG)'] * 4) / 4
    elif row['Zone_order'] == 'b':
        return round(row['Order Weight (KG)'] * 2) / 2
    elif row['Zone_order'] == 'c':
        return round(row['Order Weight (KG)'] * 1.25) / 1.25
    elif row['Zone_order'] == 'd':
        return round(row['Order Weight (KG)'] * 1.5) / 1.5
    else:
        return round(row['Order Weight (KG)'] * 2) / 2

def get_exp_charges(row):
    zone = row['Zone_order']
    weight_slab = row['Order Weight (KG)']

    # Filterring  based on zone and weight slab
    filtered_rates = rate_df[(rate_df['Zone'] == zone) & (rate_df['Weight Slabs'] == weight_slab)]

    if not filtered_rates.empty:
        fwd_fix = filtered_rates['Forward Fixed Charge'].values[0]
        add_rate = filtered_rates['Forward Additional Weight Slab Charge'].values[0]

        slab_len = 0.25 if zone == 'a' else 0.5 if zone == 'b' else 0.75 if zone == 'c' else 1.5 if zone == 'd' else 2
        add_wt = max(row['Order Weight (KG)'] - slab_len, 0)
        add_chg = add_wt * add_rate

        exp_chg = fwd_fix + add_chg
    else:

        exp_chg = 0

    return exp_chg

# Function to create a new column 'Expected Charge' in result_df
result_df['Expected Charge (Rs.)'] = result_df.apply(get_exp_charges, axis=1)

# Functions to create new columns in result_df
result_df['Order Weight Slab (KG)'] = result_df.apply(get_weight_slab, axis=1)
result_df['Expected Charge (Rs.)'] = result_df.apply(get_exp_charges, axis=1)


In [ ]:
#Comparing expected vs actual charges
result_df['Diff in Charges (Rs.)'] = result_df['Expected Charge (Rs.)'] - result_df['Billing Amount (Rs.)']

# Order level calculation
order_level_calculation = result_df[['Order ID', 'AWB Code', 'Order Weight (KG)', 'Order Weight Slab (KG)',
                                     'Charged Weight', 'Zone', 'Zone_order', 'Expected Charge (Rs.)',
                                     'Billing Amount (Rs.)', 'Diff in Charges (Rs.)']]

# Summary table
correct_count = (result_df['Diff in Charges (Rs.)'] == 0).sum()
correct_amount = result_df.loc[result_df['Diff in Charges (Rs.)'] == 0, 'Billing Amount (Rs.)'].sum()
over_count = (result_df['Diff in Charges (Rs.)'] < 0).sum()
over_amount = -(result_df.loc[result_df['Diff in Charges (Rs.)'] < 0, 'Diff in Charges (Rs.)'].sum())
under_count = (result_df['Diff in Charges (Rs.)'] > 0).sum()
under_amount = result_df.loc[result_df['Diff in Charges (Rs.)'] > 0, 'Diff in Charges (Rs.)'].sum()

c = pd.DataFrame({'Count': [correct_count, over_count, under_count],
                              'Amount (Rs.)': [correct_amount, over_amount, under_amount]},
                             index=['Total orders where X has been correctly charged',
                                    'Total Orders where X has been overcharged',
                                    'Total Orders where X has been undercharged'])
print(summary_table)


                                                 Count  Amount (Rs.)
Total orders where X has been correctly charged      0           0.0
Total Orders where X has been overcharged          124       13648.2
Total Orders where X has been undercharged           0           0.0


In [ ]:
# Save order_level_calculationto CSV
order_level_calculation.to_csv('order_level_calculation.csv', index=False)

# Save summary_table to CSV
summary_table.to_csv('summary_table.csv', index=False)


In [7]:
import pandas as pd

# Read input data from CSV files
website_order = pd.read_excel("/content/drive/MyDrive/codera/Company X - Order Report.xlsx")
pincode_mapping = pd.read_excel("/content/drive/MyDrive/codera/Company X - Pincode Zones.xlsx")
courier_invoice = pd.read_excel("/content/drive/MyDrive/codera/Courier Company - Invoice.xlsx")
sku_master = pd.read_excel("/content/drive/MyDrive/codera/Company X - SKU Master.xlsx")
charges_rate_card = pd.read_excel("/content/drive/MyDrive/codera/Courier Company - Rates.xlsx")


# Merge dataframes to get all required information
merged_df = pd.merge(website_order_df, pincode_mapping_df, left_on="Warehouse Pincode", right_on="Warehouse Pincode")
merged_df = pd.merge(merged_df, sku_master_df, left_on="SKU", right_on="SKU")
merged_df = pd.merge(merged_df, courier_invoice_df, left_on="Order ID", right_on="Order ID")

# Function to calculate applicable weight slab
def calculate_weight_slab(row):
    zone = row["Zone"]
    weight = row["Weight (g)"] / 1000  # Convert weight to kg
    slab_lengths = charges_rate_card_df[charges_rate_card_df["Zone"] == zone]["Weight Slabs"].values[0]
    for slab_length in slab_lengths:
        if weight <= slab_length:
            return slab_length

# Apply function to calculate weight slab
merged_df["Weight Slab as per X (KG)"] = merged_df.apply(calculate_weight_slab, axis=1)

# Generate output data
output_df = merged_df[["Order ID", "AWB Number", "Total weight as per X (KG)", "Weight Slab as per X (KG)",
                       "Charged Weight", "Weight slab charged by Courier Company (KG)", "Zone", "Delivery Zone",
                       "Billing Amount (Rs.)", "Expected Charge as per X (Rs.)"]]
output_df["Difference Between Expected Charges and Billed Charges (Rs.)"] = output_df["Billing Amount (Rs.)"] - output_df["Expected Charge as per X (Rs.)"]

# Generate summary table
summary_table = pd.DataFrame(columns=["Count", "Amount (Rs.)"])
summary_table.loc["Total orders where X has been correctly charged"] = [(output_df["Difference Between Expected Charges and Billed Charges (Rs.)"] == 0).sum(), 0]
summary_table.loc["Total Orders where X has been overcharged"] = [(output_df["Difference Between Expected Charges and Billed Charges (Rs.)"] > 0).sum(), output_df[output_df["Difference Between Expected Charges and Billed Charges (Rs.)"] > 0]["Difference Between Expected Charges and Billed Charges (Rs.)"].sum()]
summary_table.loc["Total Orders where X has been undercharged"] = [(output_df["Difference Between Expected Charges and Billed Charges (Rs.)"] < 0).sum(), output_df[output_df["Difference Between Expected Charges and Billed Charges (Rs.)"] < 0]["Difference Between Expected Charges and Billed Charges (Rs.)"].sum()]

# Write output to CSV
output_df.to_csv("output.csv", index=False)
summary_table.to_csv("summary_table.csv")

print("Output generated successfully.")


In [9]:
website_order = website_order.rename(columns={'ExternOrderNo': 'Order ID'})

In [10]:
# Merge datasets
merged_data = pd.merge(website_order, courier_invoice, on="Order ID")
merged_data = pd.merge(merged_data, pincode_mapping, left_on="Warehouse Pincode", right_on="Warehouse Pincode")
merged_data = pd.merge(merged_data, sku_master, on="SKU")

# Calculate total weight for X
merged_data['Total weight as per X (KG)'] = merged_data['Order Qty'] * (merged_data['Weight (g)'] / 1000)

# Calculate weight slab for X
merged_data['Weight slab as per X (KG)'] = pd.cut(merged_data['Total weight as per X (KG)'], bins=[0, 0.5, 1, 1.25, 1.5, 2], labels=['0.5', '1', '1.25', '1.5', '2'])

# Calculate total weight for courier company
merged_data['Total weight as per Courier Company (KG)'] = merged_data['Charged Weight']

# Calculate weight slab for courier company
merged_data['Weight slab charged by Courier Company (KG)'] = pd.cut(merged_data['Total weight as per Courier Company (KG)'], bins=[0, 0.5, 1, 1.25, 1.5, 2], labels=['0.5', '1', '1.25', '1.5', '2'])

# Generate summary table
summary_table = merged_data.groupby('Order ID').apply(lambda x: pd.Series({
    'Total weight as per X (KG)': x['Total weight as per X (KG)'].sum(),
    'Total weight as per Courier Company (KG)': x['Total weight as per Courier Company (KG)'].sum(),
    'Expected Charge as per X (Rs.)': x['Total weight as per X (KG)'].sum() * 100,  # Assuming Rs. 100 per KG
    'Charges Billed by Courier Company (Rs.)': x['Billing Amount (Rs.)'].sum(),
    'Difference Between Expected Charges and Billed Charges (Rs.)': (x['Total weight as per X (KG)'].sum() * 100) - x['Billing Amount (Rs.)'].sum()
})).reset_index()

# Save resultant CSV file
merged_data.to_csv("resultant_data.csv", index=False)

# Save summary table
summary_table.to_csv("summary_table.csv", index=False)

In [13]:
# Read input data from CSV files
website_order_data = pd.read_excel("/content/drive/MyDrive/codera/Company X - Order Report.xlsx")
pincode_data = pd.read_excel("/content/drive/MyDrive/codera/Company X - Pincode Zones.xlsx")
courier_company_invoice_data = pd.read_excel("/content/drive/MyDrive/codera/Courier Company - Invoice.xlsx")
sku_master_data = pd.read_excel("/content/drive/MyDrive/codera/Company X - SKU Master.xlsx")
courier_charges_data = pd.read_excel("/content/drive/MyDrive/codera/Courier Company - Rates.xlsx")


In [17]:
order_data = pd.merge(order_data, pincode_data[['Customer Pincode', 'Warehouse Pincode']], left_on='ExternOrderNo', right_on='Customer Pincode', how='left')

In [20]:
order_data

,ExternOrderNo,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001827036,8904223819093,1.0,150
2,2001827036,8904223819109,1.0,100
3,2001827036,8904223818430,1.0,165
4,2001827036,8904223819277,1.0,350
...,...,...,...,...
396,2001806229,8904223818942,1.0,133
397,2001806229,8904223818850,1.0,240
398,2001806226,8904223818850,2.0,240
399,2001806210,8904223816214,1.0,120


In [21]:

# Merge website order data with SKU master data to get weights of products
order_data = pd.merge(website_order_data, sku_master_data, on="SKU", how="left")

# Merge order data with pincode data to get delivery zone
order_data = pd.merge(order_data, pincode_data, left_on="Warehouse Pincode", right_on="Warehouse Pincode", how="left")
order_data.rename(columns={"Zone": "Delivery Zone as per X"}, inplace=True)

# Merge courier company invoice data with pincode data to get delivery zone charged by courier company
courier_invoice_data = pd.merge(courier_company_invoice_data, pincode_data, left_on="Customer Pincode", right_on="Customer Pincode", how="left")
courier_invoice_data.rename(columns={"Zone": "Delivery Zone charged by Courier Company"}, inplace=True)

# Calculate total weight as per X
order_data['Total weight as per X (KG)'] = order_data['Weight (g)'] / 1000

# Calculate weight slab as per X
order_data['Weight slab as per X (KG)'] = order_data['Total weight as per X (KG)'].apply(lambda x: round(x, 2))

# Calculate total weight as per Courier Company
courier_invoice_data['Total weight as per Courier Company (KG)'] = courier_invoice_data['Charged Weight']

# Find applicable weight slab charged by the Courier Company
def find_weight_slab_charged(row):
    weight_slabs = courier_charges_data[courier_charges_data['Zone'] == row['Delivery Zone charged by Courier Company']]
    for index, slab in weight_slabs.iterrows():
        if row['Total weight as per Courier Company (KG)'] <= slab['Weight Slabs']:
            return round(slab['Weight Slabs'], 2)
    return weight_slabs.iloc[-1]['Weight Slabs']

courier_invoice_data['Weight slab charged by Courier Company (KG)'] = courier_invoice_data.apply(find_weight_slab_charged, axis=1)

# Calculate expected charge as per X
def calculate_expected_charge(row):
    weight_slabs = courier_charges_data[courier_charges_data['Zone'] == row['Delivery Zone as per X']]
    for index, slab in weight_slabs.iterrows():
        if row['Weight slab as per X (KG)'] <= slab['Weight Slabs']:
            return slab['Forward Fixed Charge']
    return weight_slabs.iloc[-1]['Forward Fixed Charge']

order_data['Expected Charge as per X (Rs.)'] = order_data.apply(calculate_expected_charge, axis=1)

# Calculate charges billed by Courier Company
courier_invoice_data['Charges Billed by Courier Company (Rs.)'] = courier_invoice_data['Billing Amount (Rs.)']

# Calculate difference between expected charges and billed charges
courier_invoice_data['Difference Between Expected Charges and Billed Charges (Rs.)'] = courier_invoice_data['Expected Charge as per X (Rs.)'] - courier_invoice_data['Charges Billed by Courier Company (Rs.)']

# Generate the output data
result = pd.merge(order_data, courier_invoice_data, on="Order ID", how="left")
result = result[['Order ID', 'AWB Code', 'Total weight as per X (KG)', 'Weight slab as per X (KG)',
                 'Total weight as per Courier Company (KG)', 'Weight slab charged by Courier Company (KG)',
                 'Delivery Zone as per X', 'Delivery Zone charged by Courier Company',
                 'Expected Charge as per X (Rs.)', 'Charges Billed by Courier Company (Rs.)',
                 'Difference Between Expected Charges and Billed Charges (Rs.)']]

# Generate summary table
summary_table = pd.DataFrame(columns=['Count', 'Amount (Rs.)'])
summary_table.loc[0] = ['Total orders where X has been correctly charged', len(result[result['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0]), result[result['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0]['Charges Billed by Courier Company (Rs.)'].sum()]
summary_table.loc[1] = ['Total Orders where X has been overcharged', len(result[result['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0]), result[result['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0]['Difference Between Expected Charges and Billed Charges (Rs.)'].sum()]
summary_table.loc[2] = ['Total Orders where X has been undercharged', len(result[result['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0]), result[result['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0]['Difference Between Expected Charges and Billed Charges (Rs.)'].sum()]

# Save the output to CSV files
result.to_csv("result.csv", index=False)
summary_table.to_csv("summary_table.csv", index=False)

# Print the summary table
print(summary_table)

KeyError: 'Warehouse Pincode'

In [22]:
# Read input data from CSV files
website_order_report = pd.read_excel("/content/drive/MyDrive/codera/Company X - Order Report.xlsx")
pincode_mapping = pd.read_excel("/content/drive/MyDrive/codera/Company X - Pincode Zones.xlsx")
courier_company_invoice = pd.read_excel("/content/drive/MyDrive/codera/Courier Company - Invoice.xlsx")
sku_master_data = pd.read_excel("/content/drive/MyDrive/codera/Company X - SKU Master.xlsx")
courier_charges_rate_card  = pd.read_excel("/content/drive/MyDrive/codera/Courier Company - Rates.xlsx")

In [28]:
# Merge necessary dataframes
#merged_data = pd.merge(website_order_report, pincode_mapping, on="Warehouse Pincode")
merged_data = pd.merge(merged_data, sku_master, on="SKU")
merged_data = pd.merge(merged_data, courier_company_invoice, on="Order ID")

# Calculate total weight of each shipment
merged_data['Total Weight (KG)'] = merged_data['Order Qty'] * (merged_data['Weight (g)'] / 1000)

# Function to determine applicable weight slab
def calculate_applicable_weight_slab(row):
    zone_slabs = courier_charges_rate_card[courier_charges_rate_card['Zone'] == row['Zone']]
    for index, slab in zone_slabs.iterrows():
        if row['Total Weight (KG)'] <= slab['Applicable Weight']:
            return slab['Applicable Weight']
    return zone_slabs.iloc[-1]['Applicable Weight']

merged_data['Applicable Weight (KG)'] = merged_data.apply(calculate_applicable_weight_slab, axis=1)

# Calculate charges for each shipment
def calculate_charges(row):
    zone_slabs = courier_charges_rate_card[courier_charges_rate_card['Zone'] == row['Zone']]
    for index, slab in zone_slabs.iterrows():
        if row['Applicable Weight (KG)'] <= slab['Applicable Weight']:
            if row['Type of Shipment'] == 'Forward charges':
                return slab['Forward Fixed Charge']
            elif row['Type of Shipment'] == 'Forward and rto charges':
                return slab['Forward Fixed Charge'] + slab['RTO Fixed Charge']
    return None

merged_data['Expected Charges (Rs.)'] = merged_data.apply(calculate_charges, axis=1)

# Generate summary table
summary_table = pd.DataFrame(columns=['Count', 'Amount (Rs.)'])
summary_table.loc[0] = ['Total orders where X has been correctly charged', len(merged_data[merged_data['Billing Amount (Rs.)'] == merged_data['Expected Charges (Rs.)']]), merged_data[merged_data['Billing Amount (Rs.)'] == merged_data['Expected Charges (Rs.)']]['Billing Amount (Rs.)'].sum()]
summary_table.loc[1] = ['Total Orders where X has been overcharged', len(merged_data[merged_data['Billing Amount (Rs.)'] > merged_data['Expected Charges (Rs.)']]), merged_data[merged_data['Billing Amount (Rs.)'] > merged_data['Expected Charges (Rs.)']]['Billing Amount (Rs.)'].sum()]
summary_table.loc[2] = ['Total Orders where X has been undercharged', len(merged_data[merged_data['Billing Amount (Rs.)'] < merged_data['Expected Charges (Rs.)']]), merged_data[merged_data['Billing Amount (Rs.)'] < merged_data['Expected Charges (Rs.)']]['Billing Amount (Rs.)'].sum()]

# Generate output data
output_data = merged_data[['Order ID', 'AWB Code', 'Total Weight (KG)', 'Applicable Weight (KG)', 'Zone', 'Delivery Zone', 'Expected Charges (Rs.)', 'Billing Amount (Rs.)']]
output_data['Difference (Rs.)'] = output_data['Billing Amount (Rs.)'] - output_data['Expected Charges (Rs.)']

# Save data to CSV files
output_data.to_csv("output_data.csv", index=False)
summary_table.to_csv("summary_table.csv", index=False)

<ipython-input-28-be381c8e2e75>:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Weight (g)_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_data = pd.merge(merged_data, sku_master, on="SKU")
<ipython-input-28-be381c8e2e75>:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Billing Amount (Rs.)_x', 'Warehouse Pincode_x', 'Charged Weight_x', 'Type of Shipment_x', 'AWB Code_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_data = pd.merge(merged_data, courier_company_invoice, on="Order ID")


KeyError: 'Weight (g)'

In [30]:
import pandas as pd

# Load data
website_order_data = pd.read_excel("/content/drive/MyDrive/codera/Company X - Order Report.xlsx")
sku_master_data = pd.read_excel("/content/drive/MyDrive/codera/Company X - SKU Master.xlsx")


# Merge website order data with SKU master data
order_sku_data = pd.merge(website_order_data, sku_master_data, on='SKU', how='left')



In [31]:
print(order_sku_data.columns)

Index(['ExternOrderNo', 'SKU', 'Order Qty', 'Weight (g)'], dtype='object')


In [32]:
website_order_report = order_sku_data.groupby('ExternOrderNo')['SKU', 'Weight (g)'].agg(lambda x: list(x)).reset_index()

<ipython-input-32-f2dce39045c9>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  website_order_report = order_sku_data.groupby('ExternOrderNo')['SKU', 'Weight (g)'].agg(lambda x: list(x)).reset_index()


In [34]:


website_order_report.columns = ['Order ID', 'SKUs', 'Weights (g)']

# Display website order report
print("Website Order Report:")
print(website_order_report)

Website Order Report:
       Order ID                                               SKUs  \
0    2001806210                     [8904223816214, 8904223818874]   
1    2001806226                                    [8904223818850]   
2    2001806229      [8904223818706, 8904223818942, 8904223818850]   
3    2001806232                     [8904223818645, 8904223819147]   
4    2001806233                     [8904223819161, 8904223819260]   
..          ...                                                ...   
119  2001821995                     [8904223819130, 8904223818706]   
120  2001822466                     [8904223819468, 8904223819291]   
121  2001823564      [8904223819291, 8904223819031, 8904223819024]   
122  2001825261  [8904223819024, 8904223819291, 8904223818638, ...   
123  2001827036  [8904223818706, 8904223819093, 8904223819109, ...   

                                      Weights (g)  
0                                      [120, 100]  
1                                

In [35]:
import pandas as pd

# Load the invoice data

invoice_data = pd.read_excel("/content/drive/MyDrive/codera/Courier Company - Invoice.xlsx")
# Define a function to calculate expected charges
def calculate_expected_charges(row):
    # Define the courier charges rate card based on weight slabs and zones
    courier_charges_rate_card = {
        'a': {'fixed': 100, 'additional': 50},
        'b': {'fixed': 80, 'additional': 40},
        'c': {'fixed': 120, 'additional': 60},
        'd': {'fixed': 90, 'additional': 45},
        'e': {'fixed': 110, 'additional': 55}
    }

    # Extract relevant information from the invoice data row
    charged_weight = row['Charged Weight']
    warehouse_pincode = row['Warehouse Pincode']
    customer_pincode = row['Customer Pincode']
    zone = row['Zone']
    type_of_shipment = row['Type of Shipment']

    # Calculate the expected charges based on the charged weight, zone, and type of shipment
    fixed_charge = courier_charges_rate_card[zone]['fixed']
    additional_charge = courier_charges_rate_card[zone]['additional'] * (charged_weight - 1)  # Subtract 1 for the first slab
    expected_charges = fixed_charge + additional_charge

    # Return the expected charges
    return expected_charges

# Apply the calculate_expected_charges function to each row of the invoice data
invoice_data['Expected Charges'] = invoice_data.apply(calculate_expected_charges, axis=1)

# Compare actual charges with expected charges
invoice_data['Discrepancy'] = invoice_data['Billing Amount (Rs.)'] - invoice_data['Expected Charges']

# Display the invoice data with expected charges and discrepancies
print("Invoice Data with Expected Charges and Discrepancies:")
print(invoice_data)


Invoice Data with Expected Charges and Discrepancies:
          AWB Code    Order ID  Charged Weight  Warehouse Pincode  \
0    1091117222124  2001806232            1.30             121003   
1    1091117222194  2001806273            1.00             121003   
2    1091117222931  2001806408            2.50             121003   
3    1091117223244  2001806458            1.00             121003   
4    1091117229345  2001807012            0.15             121003   
..             ...         ...             ...                ...   
119  1091118551656  2001812941            0.73             121003   
120  1091117614452  2001809383            0.50             121003   
121  1091120922803  2001820978            0.50             121003   
122  1091121844806  2001811475            0.50             121003   
123  1091121846136  2001811305            0.50             121003   

     Customer Pincode Zone         Type of Shipment  Billing Amount (Rs.)  \
0              507101    d          Forw

In [38]:
import pandas as pd

# Load data
website_order_data = pd.read_excel("/content/drive/MyDrive/codera/Company X - Order Report.xlsx")
sku_master_data = pd.read_excel("/content/drive/MyDrive/codera/Company X - SKU Master.xlsx")
pincode_data = pd.read_excel("/content/drive/MyDrive/codera/Company X - Pincode Zones.xlsx")
invoice_data = pd.read_excel("/content/drive/MyDrive/codera/Courier Company - Invoice.xlsx")

website_order_data = website_order_data.rename(columns={'ExternOrderNo': 'Order ID'})

# Merge website order data with SKU master data to get total weight per order
order_weight = website_order_data.merge(sku_master_data, on='SKU', how='left').groupby('Order ID')['Weight (g)'].sum() / 1000
order_weight = order_weight.reset_index()
order_weight.columns = ['Order ID', 'Total weight as per X (KG)']

# Merge pincode data to get delivery zone
order_zone = pincode_data.set_index('Warehouse Pincode')['Zone'].to_dict()
invoice_data['Delivery Zone as per X'] = invoice_data['Warehouse Pincode'].map(order_zone)

# Calculate expected charges per order
def calculate_expected_charges(row):
    zone_charge = {
        'a': 100,
        'b': 80,
        'c': 120,
        'd': 90,
        'e': 110
    }
    weight_slab = 1.0
    charged_weight = row['Charged Weight']
    while charged_weight > weight_slab:
        weight_slab += 0.75
    expected_charge = zone_charge[row['Delivery Zone as per X']] + (charged_weight - 1) * 50
    return expected_charge

invoice_data['Expected Charge as per X (Rs.)'] = invoice_data.apply(calculate_expected_charges, axis=1)

# Create resultant dataframe
result_df = invoice_data.merge(order_weight, on='Order ID', how='left')
result_df['Weight slab as per X (KG)'] = 1.0
result_df['Weight slab charged by Courier Company (KG)'] = result_df['Charged Weight']
result_df['Delivery Zone charged by Courier Company'] = result_df['Zone']

# Rename columns
result_df.rename(columns={'AWB Code': 'AWB Number',
                          'Billing Amount (Rs.)': 'Charges Billed by Courier Company'}, inplace=True)

# Calculate difference between expected charges and billed charges
result_df['Difference Between Expected Charges and Billed Charges (Rs.)'] = result_df['Expected Charge as per X (Rs.)'] - result_df['Charges Billed by Courier Company']

# Select required columns
result_df = result_df[['Order ID', 'AWB Number', 'Total weight as per X (KG)', 'Weight slab as per X (KG)',
                       'Charged Weight', 'Weight slab charged by Courier Company (KG)', 'Delivery Zone as per X',
                       'Delivery Zone charged by Courier Company', 'Expected Charge as per X (Rs.)',
                       'Charges Billed by Courier Company', 'Difference Between Expected Charges and Billed Charges (Rs.)']]

# Save resultant dataframe to CSV file
result_df.to_csv("resultant_file.csv", index=False)

print("Resultant file saved as 'resultant_file.csv'")


Resultant file saved as 'resultant_file.csv'


In [39]:
import pandas as pd

# Load the data
invoice_data = pd.read_csv("courier_company_invoice.csv")

# Function to calculate expected charges
def calculate_expected_charges(row):
    zone_charge = {
        'a': 100,
        'b': 80,
        'c': 120,
        'd': 90,
        'e': 110
    }
    weight_slab = 1.0
    charged_weight = row['Charged Weight']
    while charged_weight > weight_slab:
        weight_slab += 0.75
    expected_charge = zone_charge[row['Zone']] + (charged_weight - 1) * 50
    return expected_charge

# Calculate expected charges
invoice_data['Expected Charge'] = invoice_data.apply(calculate_expected_charges, axis=1)

# Calculate the difference between expected charges and billed charges
invoice_data['Charge Difference'] = invoice_data['Billing Amount (Rs.)'] - invoice_data['Expected Charge']

# Summary table
summary_table = pd.DataFrame(columns=['Category', 'Count', 'Total Amount (Rs.)'])

# Calculate counts and amounts for each category
correctly_charged_count = invoice_data[invoice_data['Charge Difference'] == 0].shape[0]
correctly_charged_amount = invoice_data[invoice_data['Charge Difference'] == 0]['Billing Amount (Rs.)'].sum()
overcharged_count = invoice_data[invoice_data['Charge Difference'] > 0].shape[0]
overcharged_amount = invoice_data[invoice_data['Charge Difference'] > 0]['Charge Difference'].sum()
undercharged_count = invoice_data[invoice_data['Charge Difference'] < 0].shape[0]
undercharged_amount = invoice_data[invoice_data['Charge Difference'] < 0]['Charge Difference'].sum()

# Append values to summary table
summary_table = summary_table.append({'Category': 'Total orders where X has been correctly charged',
                                      'Count': correctly_charged_count,
                                      'Total Amount (Rs.)': correctly_charged_amount}, ignore_index=True)
summary_table = summary_table.append({'Category': 'Total Orders where X has been overcharged',
                                      'Count': overcharged_count,
                                      'Total Amount (Rs.)': overcharged_amount}, ignore_index=True)
summary_table = summary_table.append({'Category': 'Total Orders where X has been undercharged',
                                      'Count': undercharged_count,
                                      'Total Amount (Rs.)': undercharged_amount}, ignore_index=True)

# Display the summary table
print(summary_table)


                                          Category Count  Total Amount (Rs.)
0  Total orders where X has been correctly charged     0                 0.0
1        Total Orders where X has been overcharged    91              3323.0
2       Total Orders where X has been undercharged    33              -362.3


<ipython-input-39-db47797997cb>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_table = summary_table.append({'Category': 'Total orders where X has been correctly charged',
<ipython-input-39-db47797997cb>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_table = summary_table.append({'Category': 'Total Orders where X has been overcharged',
<ipython-input-39-db47797997cb>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_table = summary_table.append({'Category': 'Total Orders where X has been undercharged',


In [40]:
import pandas as pd

# Load the provided data
data = {
    'Expected Charge as per X (Rs.)': [95, 80, 155, 80, 37.5, 37.5, 80, 87.5, 55, 55, 69.5, 66, 84, 80, 37.5, 94, 55, 69.5, 40, 69.5, 73, 90, 65, 60, 79.5, 65, 70, 90, 95, 65, 110, 86.5, 60, 176, 64, 65.5, 69, 93.5, 65, 64.5, 64, 80, 88, 64, 84, 64.5, 86.5, 64.5, 64, 173, 97.5, 64, 112, 63.5, 130, 80, 105, 105, 180, 115, 69.5, 55, 68.5, 70, 68, 68, 60, 59.5, 70, 55, 177, 80, 60.5, 37.5, 40, 65, 55, 135, 40, 40, 37.5, 70, 40, 55, 60, 85, 37.5, 70, 45, 110, 65.5, 81, 59.5, 64.5, 64, 144, 64, 67, 236.5, 66.5, 82, 65, 66, 66, 64, 71, 63, 64, 123, 143.5, 64, 66, 85, 63.5, 66.5, 55, 55, 55, 55],
    'Charges Billed by Courier Company': [135, 90.2, 224.6, 61.3, 45.4, 45.4, 61.3, 89.6, 45.4, 45.4, 61.3, 90.2, 89.6, 90.2, 45.4, 135, 33, 90.2, 107.3, 61.3, 90.2, 89.6, 172.8, 102.3, 172.8, 172.8, 213.5, 258.9, 151.1, 172.8, 345, 258.9, 172.8, 174.5, 90.2, 90.2, 61.3, 135, 90.2, 90.2, 90.2, 61.3, 135, 61.3, 89.6, 90.2, 135, 90.2, 90.2, 174.5, 89.6, 90.2, 179.8, 90.2, 179.8, 61.3, 89.6, 135, 269.4, 179.8, 90.2, 45.4, 90.2, 90.2, 90.2, 90.2, 90.2, 90.2, 90.2, 86.7, 269.4, 90.2, 90.2, 86.7, 45.4, 90.2, 45.4, 224.6, 45.4, 45.4, 45.4, 90.2, 45.4, 45.4, 90.2, 135, 45.4, 61.3, 33, 117.9, 90.2, 135, 90.2, 90.2, 90.2, 224.6, 90.2, 90.2, 403.8, 90.2, 135, 90.2, 61.3, 90.2, 179.8, 224.6, 90.2, 90.2, 90.2, 90.2, 90.2, 90.2, 90.2, 179.8, 224.6, 90.2, 90.2, 135, 90.2, 90.2, 86.7, 45.4, 33, 45.4, 90.2, 90.2, 45.4, 90.2],
}

# Create a DataFrame
df = pd.DataFrame(data)

# Calculate whether X has been correctly charged, overcharged, or undercharged
df['Charge Difference'] = df['Charges Billed by Courier Company'] - df['Expected Charge as per X (Rs.)']
df['Charge Category'] = pd.cut(df['Charge Difference'], bins=[-float('inf'), 0, float('inf')], labels=['Undercharged', 'Overcharged'])

# Summary table
summary_table = pd.DataFrame(columns=['Category', 'Count', 'Total Amount (Rs.)'])

# Calculate counts and amounts for each category
correctly_charged_count = df[df['Charge Difference'] == 0].shape[0]
correctly_charged_amount = df[df['Charge Difference'] == 0]['Charges Billed by Courier Company'].sum()
overcharged_count = df[df['Charge Difference'] > 0].shape[0]
overcharged_amount = df[df['Charge Difference'] > 0]['Charge Difference'].sum()
undercharged_count = df[df['Charge Difference'] < 0].shape[0]
undercharged_amount = df[df['Charge Difference'] < 0]['Charge Difference'].sum()

# Append values to summary table
summary_table = summary_table.append({'Category': 'Total orders where X has been correctly charged',
                                      'Count': correctly_charged_count,
                                      'Total Amount (Rs.)': correctly_charged_amount}, ignore_index=True)
summary_table = summary_table.append({'Category': 'Total Orders where X has been overcharged',
                                      'Count': overcharged_count,
                                      'Total Amount (Rs.)': overcharged_amount}, ignore_index=True)
summary_table = summary_table.append({'Category': 'Total Orders where X has been undercharged',
                                      'Count': undercharged_count,
                                      'Total Amount (Rs.)': undercharged_amount}, ignore_index=True)

# Display the summary table
print(summary_table)


ValueError: All arrays must be of the same length

In [41]:
import pandas as pd

# Provided data
data = {
    'Expected Charge as per X (Rs.)': [95, 80, 155, 80, 37.5, 37.5, 80, 87.5, 55, 55, 69.5, 66, 84, 80, 37.5, 94, 55, 69.5, 40, 69.5, 73, 90, 65, 60, 79.5, 65, 70, 90, 95, 65, 110, 86.5, 60, 176, 64, 65.5, 69, 93.5, 65, 64.5, 64, 80, 88, 64, 84, 64.5, 86.5, 64.5, 64, 173, 97.5, 64, 112, 63.5, 130, 80, 105, 105, 180, 115, 69.5, 55, 68.5, 70, 68, 68, 60, 59.5, 70, 55, 177, 80, 60.5, 37.5, 40, 65, 55, 135, 40, 40, 37.5, 70, 40, 55, 60, 85, 37.5, 70, 45, 110, 65.5, 81, 59.5, 64.5, 64, 144, 64, 67, 236.5, 66.5, 82, 65, 66, 66, 64, 71, 63, 64, 123, 143.5, 64, 66, 85, 63.5, 66.5, 55, 55, 55, 55],
    'Charges Billed by Courier Company': [135, 90.2, 224.6, 61.3, 45.4, 45.4, 61.3, 89.6, 45.4, 45.4, 61.3, 90.2, 89.6, 90.2, 45.4, 135, 33, 90.2, 107.3, 61.3, 90.2, 89.6, 172.8, 102.3, 172.8, 172.8, 213.5, 258.9, 151.1, 172.8, 345, 258.9, 172.8, 174.5, 90.2, 90.2, 61.3, 135, 90.2, 90.2, 90.2, 61.3, 135, 61.3, 89.6, 90.2, 135, 90.2, 90.2, 174.5, 89.6, 90.2, 179.8, 90.2, 179.8, 61.3, 89.6, 135, 269.4, 179.8, 90.2, 45.4, 90.2, 90.2, 90.2, 90.2, 90.2, 90.2, 90.2, 86.7, 269.4, 90.2, 90.2, 86.7, 45.4, 90.2, 45.4, 224.6, 45.4, 45.4, 45.4, 90.2, 45.4, 45.4, 90.2, 135, 45.4, 61.3, 33, 117.9, 90.2, 135, 90.2, 90.2, 90.2, 224.6, 90.2, 90.2, 403.8, 90.2, 135, 90.2, 61.3, 90.2, 179.8, 224.6, 90.2, 90.2, 90.2, 90.2, 90.2, 90.2, 90.2, 179.8, 224.6, 90.2, 90.2, 135, 90.2, 90.2, 86.7, 45.4, 33, 45.4, 90.2, 90.2, 45.4, 90.2],
}

# Create DataFrame
df = pd.DataFrame(data)

# Calculate charge difference
df['Charge Difference'] = df['Charges Billed by Courier Company'] - df['Expected Charge as per X (Rs.)']

# Initialize summary table
summary_table = pd.DataFrame(columns=['Category', 'Count', 'Total Amount (Rs.)'])

# Count and sum for correctly charged, overcharged, and undercharged
correctly_charged_count = df[df['Charge Difference'] == 0].shape[0]
correctly_charged_amount = df[df['Charge Difference'] == 0]['Charges Billed by Courier Company'].sum()

overcharged_count = df[df['Charge Difference'] > 0].shape[0]
overcharged_amount = df[df['Charge Difference'] > 0]['Charge Difference'].sum()

undercharged_count = df[df['Charge Difference'] < 0].shape[0]
undercharged_amount = df[df['Charge Difference'] < 0]['Charge Difference'].sum()

# Add data to summary table
summary_table = summary_table.append({'Category': 'Total orders where X has been correctly charged',
                                      'Count': correctly_charged_count,
                                      'Total Amount (Rs.)': correctly_charged_amount}, ignore_index=True)
summary_table = summary_table.append({'Category': 'Total Orders where X has been overcharged',
                                      'Count': overcharged_count,
                                      'Total Amount (Rs.)': overcharged_amount}, ignore_index=True)
summary_table = summary_table.append({'Category': 'Total Orders where X has been undercharged',
                                      'Count': undercharged_count,
                                      'Total Amount (Rs.)': undercharged_amount}, ignore_index=True)

# Print summary table
print(summary_table)


ValueError: All arrays must be of the same length